# Suppy Chain Dataset Analysis 
---
<B> Prompt For Chat GPT </B> <br>
Hello , I am trying to create a python code which will help me generate data. I am looking to generate a data for Supply Chain. This is a data for a warehouse which have a information of orders that is incoming everyday. These are the values I am looking for and I have also provided the information each variable should have :
*order_id – unique ID for each order ( random numerical). This should be unique for each item 
* item_category – Electronics, Apparel, Office Supplies, Food & Beverage, Medical Supplies.
* supplier_name – Supplier_A, Supplier_B, Supplier_C.
* Warehouse_name - Name of Warehouse (create random name of Business name for should have characters have a-Z, A-Z and numerical values 1-9). This doesn’t have to be unique and each warehouse have a location it is associated with . 
* warehouse_location – Generate Random location for each unique warehouse should be associated with a warehouse_name.
* fc_associate – ID or name of associate who processed the order.
* order_status – Pending, Received, Backordered, Canceled.
* total_quantity_required – how many units were ordered.
* total_quantity_received – how many units actually arrived.
* total_quantity_in_transfer – units still pending shipment (required – received).
* unit_price – price per unit (varies by category).
* order_cost – total_quantity_received × unit_price.
* lead_time_days – how many days from order to delivery. ( make it random so, I want couple of data where need by date > recived date and also couple of time where recived date is null)
* need_by_date – deadline for when the order is needed. (I want couple of data where need by date > recived date and have some need by date > today and some less than todayb with  receive_date as null)
* receive_date – when the items were actually received. ( make it random so, I want couple of data where need by date > recived date and also couple of time where recived date is null) 
---

<B> Data Generation Code for Supply Chain </B>

In [243]:
import pandas as pd
import numpy as np
import random
from faker import Faker

# Initialize
fake = Faker()
np.random.seed(42)
random.seed(42)

# Parameters
n = 2000  # number of rows

item_categories = {
    "Electronics": (50, 500),
    "Apparel": (10, 100),
    "Office Supplies": (5, 50),
    "Food & Beverage": (2, 20),
    "Medical Supplies": (20, 200)
}
suppliers = ["Supplier_A", "Supplier_B", "Supplier_C"]
statuses = ["Pending", "Received", "Backordered", "Canceled"]

# Generate warehouse names (ensuring repetition)
warehouses = [f"WH_{fake.lexify(text='???').upper()}{random.randint(1,9)}" for _ in range(10)]
warehouse_locs = {w: fake.city() for w in warehouses}

data = []
for i in range(n):
    order_id = fake.unique.random_int(10000, 99999)
    category = random.choice(list(item_categories.keys()))
    supplier = random.choice(suppliers)
    warehouse = random.choice(warehouses)
    warehouse_loc = warehouse_locs[warehouse]
    fc_associate = f"Assoc_{random.randint(100,999)}"
    
    status = random.choice(statuses)
    
    qty_required = random.randint(10, 200)
    qty_received = qty_required if status == "Received" else random.randint(0, qty_required)
    qty_in_transfer = qty_required - qty_received
    
    unit_price = random.randint(*item_categories[category])
    order_cost = qty_received * unit_price
    
    # Dates
    order_date = fake.date_between(start_date="-60d", end_date="today")
    need_by_date = order_date + pd.Timedelta(days=random.randint(3, 15))
    
    if status == "Received":
        lead_time = random.randint(1, 20)
        receive_date = order_date + pd.Timedelta(days=lead_time)
        # enforce ~70% late deliveries
        if random.random() < 0.7:
            receive_date = need_by_date + pd.Timedelta(days=random.randint(1, 7))
    else:
        lead_time = None
        receive_date = None
    
    data.append([
        order_id, category, supplier, warehouse, warehouse_loc, fc_associate,
        status, qty_required, qty_received, qty_in_transfer,
        unit_price, order_cost, lead_time, need_by_date, receive_date
    ])

# Build DataFrame
columns = [
    "order_id","item_category","supplier_name","warehouse_name","warehouse_location",
    "fc_associate","order_status","total_quantity_required","total_quantity_received",
    "total_quantity_in_transfer","unit_price","order_cost","lead_time_days",
    "need_by_date","receive_date"
]

df = pd.DataFrame(data, columns=columns)

# Export to CSV
df.to_csv("warehouse_orders.csv", index=False)

print("Generated 2000 rows and saved to warehouse_orders.csv")
print(df.head())


Generated 2000 rows and saved to warehouse_orders.csv
   order_id     item_category supplier_name warehouse_name warehouse_location  \
0     50931       Electronics    Supplier_A        WH_WEY1     Port Garyburgh   
1     52498  Medical Supplies    Supplier_B        WH_MRB4         Knightland   
2     88895   Office Supplies    Supplier_B        WH_IGS5        Patrickside   
3     65846   Office Supplies    Supplier_B        WH_YXR7         Brendabury   
4     66053   Food & Beverage    Supplier_A        WH_GYU2        Katelynside   

  fc_associate order_status  total_quantity_required  total_quantity_received  \
0    Assoc_323     Received                      139                      139   
1    Assoc_559  Backordered                       11                        2   
2    Assoc_320  Backordered                       36                        5   
3    Assoc_370      Pending                      196                      117   
4    Assoc_400  Backordered                      157  

--
<B> Understading Data </B>
* I wanted to check the starting and ending rows of the data so I added the following lines of code. This is so I can check what type of data were generated. 


In [245]:
# I wanted to check the starting and ending rows of the data so I added the following lines of code. 
print("These are the starting 10 rows for the data \n",df.head(10))
print("These are the ending 5 rows for the data \n",df.tail(10))


These are the starting 10 rows for the data 
    order_id     item_category supplier_name warehouse_name warehouse_location  \
0     50931       Electronics    Supplier_A        WH_WEY1     Port Garyburgh   
1     52498  Medical Supplies    Supplier_B        WH_MRB4         Knightland   
2     88895   Office Supplies    Supplier_B        WH_IGS5        Patrickside   
3     65846   Office Supplies    Supplier_B        WH_YXR7         Brendabury   
4     66053   Food & Beverage    Supplier_A        WH_GYU2        Katelynside   
5     86416           Apparel    Supplier_B        WH_WEY1     Port Garyburgh   
6     20330           Apparel    Supplier_B        WH_INM3  Port Kathrynshire   
7     66272  Medical Supplies    Supplier_C        WH_IGS5        Patrickside   
8     49114  Medical Supplies    Supplier_A        WH_INM3  Port Kathrynshire   
9     93217   Office Supplies    Supplier_A        WH_MRB4         Knightland   

  fc_associate order_status  total_quantity_required  total_qu

Similarly I wanted to check : 
* I want to check the dimensions of the DataFrame so df.shape (Output : 2000 rows and 15 columns)
* Second I wanted to check the column names of the DataFrame and the number of columne name  (Output : It has listed all the 15 column names)
* Third, I wanted to check the data types of each column in the DataFrame (Output : It has listed all the 15 column names with their respective data types)
* I also wanted to check the total number of elements in the DataFrame (Output : 30000 which is 2000*15)
* I also wanted to the number of dimensions of the DataFrame (Output : 2 which means it is a 2D array)
* Finally, I though to get  summary of the DataFrame, including non-null counts and memory usage
--
My starting point of my analysis was to check if the any warehouses that are at risk where we are waiting for pending items and that are overdue. So, to start with this analysis I started with checking the unique warehouse and used describe methods to get a summary of the 'warehouse_name' column, including count, unique values, top value, and frequency


In [246]:
# I want to understand the data better so I added the following lines of code.
df.shape # To check the dimensions of the DataFrame (Output : 2000 rows and 15 columns)
df.columns # To check the column names of the DataFrame (Output : It has listed all the 15 column names)
df.dtypes # To check the data types of each column in the DataFrame (Output : It has listed all the 15 column names with their respective data types)
df.size # To check the total number of elements in the DataFrame (Output : 30000 which is 2000*15)
df.ndim # To check the number of dimensions of the DataFrame (Output : 2 which means it is a 2D array)

df.info() # To get a concise summary of the DataFrame, including non-null counts and memory usage

df['warehouse_name'].unique() # To get the unique values in the 'warehouse_name' column
df['warehouse_name'].describe() # To get a summary of the 'warehouse_name' column, including count, unique values, top value, and frequency


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   order_id                    2000 non-null   int64  
 1   item_category               2000 non-null   object 
 2   supplier_name               2000 non-null   object 
 3   warehouse_name              2000 non-null   object 
 4   warehouse_location          2000 non-null   object 
 5   fc_associate                2000 non-null   object 
 6   order_status                2000 non-null   object 
 7   total_quantity_required     2000 non-null   int64  
 8   total_quantity_received     2000 non-null   int64  
 9   total_quantity_in_transfer  2000 non-null   int64  
 10  unit_price                  2000 non-null   int64  
 11  order_cost                  2000 non-null   int64  
 12  lead_time_days              490 non-null    float64
 13  need_by_date                2000 

count        2000
unique         10
top       WH_INM3
freq          231
Name: warehouse_name, dtype: object

Once, I have the information of the warehouse , I wanted to chect how make order was places for each warehouse. 
Once, this was completed , I wanted to check which warehouse is having the most lead time meaning which warehouse is unable to recieve the items on time. As, items not being delived on time can cause delay in the projects and impact. 
This anslysis can help initally find the warehouse which is having issues with the items not being delivered on time. So, I used pandas to find the min, max, mean leadtime with the total order and total purchase order. 

In [247]:

# To check how many orders were placed from each warehouse 
df['warehouse_name'].value_counts()
# I also wanted the location of each warehouse so I added the following line of code. 
df.groupby(['warehouse_name', 'warehouse_location']).size().reset_index(name='count')
# To check the average lead time days for each warehouse . To get information about how efficient each warehouse is in terms of lead time.
df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].mean().round(2).reset_index()

df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].mean().round(2).reset_index()

#.agg() - I wanted to find the max and min leadtime days so it will help me with analysing the ware house efficiently 
df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].agg(['min','max','mean']).round(3)

warehouse_summary = df.groupby(['warehouse_name', 'warehouse_location']).agg(min_lead_time_delay=('lead_time_days', 'min'),max_lead_time_delay=('lead_time_days', 'max'),
                                                                             mean_lead_time_delay=('lead_time_days', 'mean'),
                                                     total_orders=('order_id', 'count'), total_purchase_amount=('order_cost', 'sum')).round(3).reset_index()
warehouse_summary



,warehouse_name,warehouse_location,min_lead_time_delay,max_lead_time_delay,mean_lead_time_delay,total_orders,total_purchase_amount
0,WH_CJW2,Angelaton,1.0,20.0,9.974,176,940617
1,WH_GYU2,Katelynside,1.0,19.0,9.529,174,1195053
2,WH_IGS5,Patrickside,2.0,20.0,11.074,224,1323859
3,WH_INM3,Port Kathrynshire,1.0,19.0,9.571,231,1389104
4,WH_KSX2,Lake Henry,1.0,20.0,11.463,194,1238664
5,WH_MRB4,Knightland,1.0,20.0,11.241,188,1522715
6,WH_NVC4,Port Kyle,1.0,20.0,9.940,175,829103
7,WH_SFH9,East Evanport,1.0,20.0,12.118,195,979262
8,WH_WEY1,Port Garyburgh,1.0,20.0,11.534,222,1551520
9,WH_YXR7,Brendabury,2.0,20.0,11.667,221,1336753


Once this values were created, I sorted the values in acending for mean lead delay so, I can easily find the highest lead time delays. Then I wanted to check the overdue items so, I wrote a query to check on the overdue items. These items are still on pending and has not been recieved yet. 

In [248]:
# Now I want to sort by the total orders and mean lead time for order so that I can see which warehouse has the most orders and how efficient they are in terms of lead time.
warehouse_summary.sort_values(by=['mean_lead_time_delay'], ascending=[False],ignore_index=True,inplace=True)
# Going back to df for more analysis

import datetime

today = datetime.date.today()  # gets today's date at midnight

df['order_status'].value_counts() # To see the distribution of different order statuses in the dataset
df.groupby(['warehouse_name', 'order_status']).size()
df['is_overdue'] = (
    (df['order_status'] == "pending")
    & df['receive_date'].isna()
    | (df['need_by_date'] < df['receive_date'])
)

df['is_overdue']

status_counts = (
    df.groupby(['warehouse_name', 'is_overdue'])
      .size()
      .unstack(fill_value=0)
      .reset_index()
      .rename(columns={True: 'overdue_orders', False: 'not_overdue_orders'})
)

warehouse_summary = warehouse_summary.merge(
    status_counts,
    on='warehouse_name',
    how='left'
)
warehouse_summary


,warehouse_name,warehouse_location,min_lead_time_delay,max_lead_time_delay,mean_lead_time_delay,total_orders,total_purchase_amount,not_overdue_orders,overdue_orders
0,WH_SFH9,East Evanport,1.0,20.0,12.118,195,979262,151,44
1,WH_YXR7,Brendabury,2.0,20.0,11.667,221,1336753,169,52
2,WH_WEY1,Port Garyburgh,1.0,20.0,11.534,222,1551520,170,52
3,WH_KSX2,Lake Henry,1.0,20.0,11.463,194,1238664,157,37
4,WH_MRB4,Knightland,1.0,20.0,11.241,188,1522715,140,48
5,WH_IGS5,Patrickside,2.0,20.0,11.074,224,1323859,175,49
6,WH_CJW2,Angelaton,1.0,20.0,9.974,176,940617,142,34
7,WH_NVC4,Port Kyle,1.0,20.0,9.940,175,829103,132,43
8,WH_INM3,Port Kathrynshire,1.0,19.0,9.571,231,1389104,186,45
9,WH_GYU2,Katelynside,1.0,19.0,9.529,174,1195053,144,30


Finally, I wanted to find the warehouse with the highest number of order pending to be delivered and the have the highest lead time so, I can find the warehouse which has the struggling the most.  

In [256]:
warehouse_summary['not_delivered'] = warehouse_summary['overdue_orders'] / warehouse_summary['total_orders']
warehouse_summary['warehouse_delivery_status'] = pd.cut(
    warehouse_summary['not_delivered'],       # column to categorize
    bins=[-1, 0.10, 0.20, 1],      # define intervals
    labels=['Green', 'Yellow', 'Red']  # category labels
)
warehouse_summary.sort_values(by=['mean_lead_time_delay','warehouse_delivery_status'], ascending=[False,True],ignore_index=True,inplace=True)
warehouse_summary

,warehouse_name,warehouse_location,min_lead_time_delay,max_lead_time_delay,mean_lead_time_delay,total_orders,total_purchase_amount,not_overdue_orders,overdue_orders,not_delivered,warehouse_delivery_status
0,WH_SFH9,East Evanport,1.0,20.0,12.118,195,979262,151,44,0.225641,Red
1,WH_YXR7,Brendabury,2.0,20.0,11.667,221,1336753,169,52,0.235294,Red
2,WH_WEY1,Port Garyburgh,1.0,20.0,11.534,222,1551520,170,52,0.234234,Red
3,WH_KSX2,Lake Henry,1.0,20.0,11.463,194,1238664,157,37,0.190722,Yellow
4,WH_MRB4,Knightland,1.0,20.0,11.241,188,1522715,140,48,0.255319,Red
5,WH_IGS5,Patrickside,2.0,20.0,11.074,224,1323859,175,49,0.218750,Red
6,WH_CJW2,Angelaton,1.0,20.0,9.974,176,940617,142,34,0.193182,Yellow
7,WH_NVC4,Port Kyle,1.0,20.0,9.940,175,829103,132,43,0.245714,Red
8,WH_INM3,Port Kathrynshire,1.0,19.0,9.571,231,1389104,186,45,0.194805,Yellow
9,WH_GYU2,Katelynside,1.0,19.0,9.529,174,1195053,144,30,0.172414,Yellow
